# Feature Learning with Raw-Waveform CLDNNs for Voice Activity Detection

VAD는 ASR의 preprocessing으로 주로 사용되어 왔다.

이전까지는 DNN 기반의 VAD가 주로 사용되어 왔으며 좋은 성능을 보여왔다. 하지만 CNN, LSTM 기반의 VAD가 등장하며 VAD의 모델이 이들로 교체됐는데 그 이유는 CNN, LSTM이 DNN보다 좋은 성능을 보여주었기 때문이다.

DNN, CNN, LSTM 모델은 각각 서로를 보완할 수 있는 장점이 있는데 다음과 같다.
1. CNN
    - CNN은 frequency variation을 줄여준다.(feature 뽑을 때 convolution을 통해서 feature variance를 줄인다는 뜻(?))

2. LSTM
    - LSTM은 sequential한 데이터를 다루는데 장점이 있다.
    - 음성을 결국 sequential한 데이터이므로 LSTM 모델과 잘 맞는다.

3. DNN
    - feature를 분리하여 매핑하는 것에 장점이 있다.

위와 같이 각각의 모델들이 서로 보완적인 특성을 가지므로 위 세 모델을 결합한 Convolutional, Long Short-Term Memory, Fully Connected Deep Neural Networks(CLDNN) 모델이 제안되었다.<br>
[->CLDNN 논문](https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/43455.pdf)

본 논문의 저자는 VAD는 sequential한 데이터니깐 DNN보다 temporal한 모델이 더 좋을거라고 굳게 믿고 있어서 CLDNN과 같은 모델들을 DNN과 비교하려고 한다.

또한 기존 모델들은 mel-scale의 형식을 같는 feature를 뽑아서 사용하였는데, mel-scale은 확실히 사람이 소리를 듣는 메커니즘을 모방하였기 때문에 음성 관련 작업들에 대해 좋은 성능을 보여줬다. 하지만 VAD와 noise-detection은 ASR과 같은 speech process task와 약간 다르다는게 저자들의 생각인거 같다. 그래서 본 논문에서는 CLDNN based VAD 외에도 raw waveform을 이용한 CLDNN based VAD 모델도 제안하였다.

## Architectures

1. DNN
- k hidden layer, n hidden units per layer
- 각 hidden layer마다 ReLU 사용
- softmax 사용
- input: 40 dimensional log-mel scale feature every 10ms, 이 때 앞뒤로 5 frame을 포함하여 window를 잡음(context를 위해)

![DNN](https://i.imgur.com/Q54APeZ.png)

2. LSTM
- sequential model
- k hidden layer, n hidden units per layer
- projection layer를 사용하지 않는다.
- input: 40 dimensional log-mel scale feature
![LSTM](https://i.imgur.com/h6rqbNA.png)


3. CLDNN
- sequential model
- CNN으로 frequency variation을 줄이고 LSTM으로 시간적 데이터를 사용하고 DNN을 통해 feature를 mapping한다. 3개의 모델의 장점을 모두 사용하겠다는 의도이다. 순서는 CNN에서 feature를 다루고 LSTM으로 보낸 뒤 DNN에서 마지막으로 처리한다.
- input: 40 dimensionanl log-mel scale feature
- convolutional layer: 
    - filter size: TIME X FREQUENCY
    - pooling: non-overlapping max pooling along the frequency axis
- LSTM layer
- DNN layer
![CLDNN](https://i.imgur.com/e6vsdW1.png)


4. Raw Waveform CLDNN
- CLDNN이 "Learning the speech front-end with raw waveform CLDNNs"에서 log-mel feature와 비슷하게 frequency를 학습한다는 것이 밝혀졌고 noisy한 환경에서 성과가 있었다. 따라서 본 논문에서 VAD에도 좋은 결과가 나오는지 확인해 볼 것이다.
- 데이터는 일정 길이의 raw waveform sample을 사용한다.
- M: input sample size, P: 필터 개수, N: filter size
![Raw waveform CLDNN](https://i.imgur.com/quXaV27.png)


## 실험
1. Dataset
    1. training
    - 3,800 hours(3 million uterance)
    - clean data에 youtube 등 다양한 출처의 noise를 다양한 snr level로 합성해서 만든 데이터셋
    - speech, nonspeech 비율은 반반
    2. test
        1. noisy test set
        - 20 hours
            - 3 hours: noisy speech
            - 15 hours: noisy background
        2. clean test set
        - 30 hours

2. Training
- optimizer: asynchronous stochastic gradient descent
- criterion: cross entropy
- 초기화: CNN, DNN에 Glorot-Bengio strategy, LSTM에는 -0.02~0.02값 랜덤하게.
- learning rate: exp decay

## Result

결론부터 말하자면 rawcldnn과 cldnn은 다른 모델에 비해 성능이 더 좋다. 이는 clean, noisy test set 둘 다 해당되는데 rawcldnn은 noisy한 테스트 환경일 때 더욱 성능이 좋다.
![result](https://i.imgur.com/BKS4kod.png)

위 결과를 보면 CLDNN 모델이 DNN 모델에 비해 smooth한 곡선의 형태를 띄는 것을 볼 수 있다. 이는 모델의 temporal한 성격이 context를 이해하기 때문에 띄는 특성으로 볼 수 있다.

![filterbank](https://i.imgur.com/4pTbtpS.png)

모델의 다른 특성으로는 mel filterbank와 비교했을때 rawcldnn이 더 높은 frequency를학습한 형태를 보인다. 이는 VAD에서 speech와 noise를 구분하는 것은 보통 acoustic speech process에서 사용하는 mel scale보다 더 높은 주파수 대역을 사용하는 것이 유리하다는 것을 알 수 있다.

모델의 사이즈, 파라미터 수에 따른 FA는 모델과 파라미터가 커질수록 성능이 좋아지는 경향을 보이지만 raw cldnn에서 noisy한 테스트환경에서만 미세하게 성능이 떨어지는 것을 보여준다. 하지만 본 논문에서는 그래도 기존 cldnn보다 noisy한 환경에서 훨씬 좋은 성능을 보여주므로 별 문제 아닌 것처럼 본다.



+ 추가적으로 생각해볼만한 것은 raw cldnn의 결과물은 uterance의 끝이 다른 모델에 비해 더 짧다. 이는 raw input을 사용하기 때문에 말 끝의 에너지가 약한 특성 때문에 그럴수도 있지 않을까란 생각을 해봄.